# Table of content
1. [Learning to predict the next word](#intro)  
1.1. [A relational learning task](#rel)  
1.2. [A large-scale example](#large-scale)
2. [A brief diversion into cognitive science](#cogsci)
3. [Another diversion: The softmax output function](#softmax)  
3.1. [Problems with squared error](#problem)  
3.2. [Softmax](#softmax)  
3.3. [Cross-entropy: the right cost function to use with softmax](#cross-enp)

# 1. Learning to predict the next word
<a id="intro"></a>
* Aim: Using backpropagation algorithm to learn the feature representation of the meaning of the word.
* Example: simple one, but slow that shows idea about how can take some relational information, then use backpropagation algorithm to turn relational information into feature vectors that capture the meanings of words.

## 1.1. A relational learning task
<a id="rel"></a>
- A simple example is a diagram shows a simple family tree (a simple example of relational information).
- Task: use backpropagation neural network to understand the information in this family tree.
- The first diagram ($\color{black}{top}$) is English family tree, and the second diagram ($\color{red}{bottom}$) is Italian family tree.
![family tree](images/family_tree.png?raw=true)
- The information of these family trees can be expressed as a set of propositions (naming the relationships in family trees and using those relationships to write down a set of triples)
- The relational learning task is defined the task of learning the information in those family trees, can be viewed as figuring out the **regularities** in a **large set of triples** that express the information in those trees
 - The obvious way to express the regularities is **symbolic rules**
   - Ex: $(x\ has-mother\ y)\ \&\ (y\ has-husband\ z)\rightarrow(x\ has-father\ z)$
 - Can search the such rules, but this would involve a search through quite a large space, a combinationally large space of discrete possibilities. (Finding the symbolic rules involves a difficult search through a very large discrete space of possibilities)

- Another way is to use a neural network that searches through a continuous space of real-valued weights to try and capture the information.
- The way that a neural net capturing the information (The structure of the neural net) = can predict the third terminal triple from the first two terms
![neural net structure](images/neural_net_structure.png?raw=True)  
![neural net family ties](images/neural_net_family_ties.png?raw=True)  
The input layer is divided in two parts: $24$ cells represent the first person, $12$ cells represent the relation.
The output layer is composed of $24$ cells representing the second person.
The learning phase ($3$ intermediate layers) uses the backpropagation algorithm, with $112$ relations as training example.
**Explanation**: At the bottom, putting in a person and a relationship, and the information is going to flow **forwards** through this neural network. What we're going to try to get out of this neural network after it's learned is the person who's related to the first person by that relationship.  
* _Step 1_: What we do is we encode the information in a neural way:  
    - There're $24$ possible people $\rightarrow$ the block at the bottom (local encoding of person $1$) has $24$ neurons, and exactly one of those will be turned **on** for each training case.  
    - Similarly, there're $12$ relationships, and exactly one of the relationship units will be turned **on**.  
**Question for encoding the person:**
![question 1 p1](images/question_1.PNG?raw=True)
![question 1 p2](images/question_2.PNG?raw=True)
* _Step 2_: Then, for a relationship that has a **unique** answer, we would like one of the $24$ people at the top (one of the $24$ output people to turn **on** to represent the answer.  
(By using a representation in which exactly **one** of the neurons is **on** $\rightarrow$ don't accidentally give the network any similarities between people. All pairs of people are equally dissimilar $\rightarrow$ not cheating by the information network about who's like who, so as far as the network is concerned, the people are uninterpreted symbols.)
* _Step 3_: The second layer of the network has $6$ possible values, so the $1$-high-bit $24$ bit input vector is projected into a less sparse smaller representation (taking the local encoding of person $1$, and connecting it to a small set of neurons ($6$), due to $24$ people, can't possibly dedicate $1$ neuron to each person $\rightarrow$ to re-present the people as patterns of activity over those $6$ neurons ($6$ feature groups including such as $1st$ hidden cell can be interpreted as representing the nationality (_English, Italian_) of the input person: the weights connecting this cell to two input cells coding two persons at the same place in the both trees are **opposite**, $2nd$ hidden unit encodes the generation, $3rd$ hidden cell is the age of the person (_old, medium, young_), $4th$ hidden cell encodes their sex (_M/F_), and $6th$ encodes the branch of the family (which branch of the family tree a person belonged in (_left, middle or right_)). These features are like those which the hidden layers are supposed to discover, and are used to illustrate how easy the mapping task is when these features are explicitly present in the training environment. With $12$ relationships, we have $6$ neurons, which are _son, daughter, nephew, niece; father, mother, uncle, aunt; brother, sister, husband, wife_  
        * This leads to description of structure of the task.
        * The discussion of this is unclear but claim is that one bit represents whether the person is from the English or Italian people (they are disjoint in their relationship, but identical in their relationship structure).
        * The second set of weights represents the generation of the person.
        * The third weight encodes which half of the tree the person lives in.

The networks had $60$ hidden units in the coding and association hidden layers, whereas only $6$ units in the coding layers and $12$ in the association hidden layer. Then, analyze the weights between the input layer and the hidden layers, that is possible to extract semantic information from the neural network (through $112$ propositions).    
$6$ hidden units as those $6$ big gray blocks, laying out $24$ people with $12$ English people in the row along the top, and $12$ Italian people in the row underneath $\rightarrow$ $24$ blobs in each of $6$ blocks of hidden cells, which is the incoming weights for one of the hidden units in the **distributed encoding** layer - weights from layer $1$ to layer $2$  
For example, as you can see the first top-left gray rectangular block in the following image, the weights of the top row is always positive, and the weights of the bottom row is always negative, which determines the input person is English or Italian (due to the input person is English, then the output person is also English) $\rightarrow$ allow to predict one bit of information about the output.  
![english family tree](images/english_family_tree.png?raw=True)

Looking at the gray block immediately below that, the second one down on the right, there are $4$ big positive weights for each $12$ people, which is _Christopher, Andrew, Penelope and Christine_ and their Italian equivalents, also $2$ big negative weights for each, which is _Colin and Charlotte_ and their Italian equivalents. $\rightarrow$ represents what generation somebody is (big positive weights to the oldest generation, big negative weights to the youngest generation, and intermediate weights which are roughly zero to the intermediate generation $\rightarrow$ a three-value feature, the generation of the person)
![generation](images/generation.png?raw=True)

Finally, looking at the bottom gray rectangle on the left-hand side, there is a different structure. As you can see in the negative weights in the top row of that unit, which are _Andrew, James, Charles, Christine and Jennifer_ (are all in the right-hand branch of the English family tree $\rightarrow$ that unit has learned to represent which branch of the family tree someone is in (very useful feature to have for predicting the output person, because if it's a close family relationship, you expect the output to be in the **same** branch of the family tree as the input).
![branch](images/branch.png?raw=True)

When all three are on, these units pick out _Christopher_ and _Penelope_. Other combinations pick out other parts of the trees.  
![total family tree predict](images/total_family_predict.PNG?raw=True)

Similarly, there is a same procedure for relationship units  
![relations](images/relation.png?raw=True)  
*Note*: If you want to investigate more about Hilton's neural network used in family tree learning, you can search the papers in the _Google_ with its title: _**Learning Distributed Representations of Concepts using
Linear Relational Embedding (Alberto Paccanaro, Geoffrey E. Hinton)**_ and _**Learning distributed representations of concepts (Geoffrey E. Hilton) in Proceedings of the Eighth Annual Conference of the Cognitive Science Society, Amherst, Mass. 1986, pages 1-12**_

* What the network learns
    - The $6$ hidden units in the bottleneck connected to the input representation of person $1$ learn to represent features of people that are useful for predicting the answer. These features are such as _nationality, the generation, and branch of the family tree_ that are good features for expressing the regularity in this domain.  
    - Of course, these features are only useful, if the other bottlenecks (the one for relationships) use similar representations and the central layer is able to say how the features of the input person and the features of the relationship predict the features of the output person (i.e. learns how features predict other features).  
    - For example, if  
        - input person is of generation $3$ $\color{red}{and}$  
          relationship requires answer to be one generation up  
          $\color{red}{implies}$  
        output person is of generation $2$
        - But **notice** to capture that rule, you have to extract appropriate features at the first hidden layer, and the last hidden layer of the network, and have to make the units in the middle, related to those features correctly.  
* Another way to see that it works  
- Train the network on all, but $4$ of the triples that can be made using $12$ relationships (i.e. $4$ training cases for testing) and see if it can complete those triples correctly, does it generalize? (there're $112$ triples, training $108$ of them, and testing on the remaining $4$ cases, also doing it **several times**. Getting either $second$ or $third$ of those right $\rightarrow$ not so bad for $24$ way choice, so it's true making mistakes, but it didn't have much training data, there's not enough triples in this domain to really nail down (pin down) the regularities very well, and it does much better than chance.  

## 1.2. A large-scale example
<a id="large-scale"></a>
- Suppose having a database of millions of relational facts of the form $(A\ R\ B)$ - $A\ has\ relationship\ R\ to\ B$ $\rightarrow$ training a net to discover feature vector representations of the terms ($A$ and $R$) that allow the third term ($B$) to be predicted from the first two.  
- Then use the trained net to find very unlikely triples which are good candidates for errors in the database
- Instead of predicting the third term, use all $3$ terms as input and predict the probability that the fact is correct.

# 2. A brief diversion into cognitive science
<a id="cogsci"></a>

# 3. Another diversion: The softmax output function
<a id="softmax"></a>
- This's a way of forcing the outputs of a neural network to sum to $1$ $\rightarrow$ they can represent a probability distribution across discrete, mutually exclusive alternatives.

## 3.1. Problems with squared error
<a id="problem"></a>
- If the desired output is very large ($1$ for example in binary threshold unit) and the actual output is very small compared to the desired output (i.e. $0.00000001$), there is almost **no gradient** for **a logistic unit** to fix up the error. (due to the error derivative w.r.t. the weights $\frac{\partial E}{\partial w_i}$ is very small $\rightarrow$ almost no gradient.)  
$\rightarrow$ very long, long time to change its weights, and it also make almost as big an error as it's possible.
- If trying to assign probabilities to mutually exclusive class labels (các nhãn lớp riêng biệt) (i.e. each entry is in exactly one class), knowing that the output should be summed to $1$, but we are depriving the network of this knowledge ($?$) (it is be advise to tell the network this information (the sum of matually exclusive class probabilities is $1$), but with squared error, this information is deprived).  
$\rightarrow$ is there a different **cost function** that will work better?  
Is there a way of telling it that these are mutually exclusive and then, using an appropriate cost function?  
- The answer is yes, what we need to do is force the outputs of the neural net to represent a probability distribution across discrete alternatives.

![question abt squared errors's issue](images/question_5.PNG?raw=True)  
![question abt squared errors's issue 1](images/question_6.PNG)

## 3.2. Softmax
<a id="softmax"></a>
- The way we going to use to force the outputs of the neura network is a thing called the _soft-max_.
- It's a kind of soft continuous version of the maximum function
- The way the units in a soft-max group work is that they each receive some total input they've accumulated from the layer below, that is $z_i$ for the $i-th$ unit (called **the logit**), and they give **the output** $y_i$ (depended on $z_i$ from itself and $Z$s accumulated by their lines as well)  
The output units in a softmax group use a non-local non-linearity:
![output softmax](images/output_softmax.png?raw=True)
$$\rightarrow y_i=\frac{exp(z_j)}{\sum_{j\in group}{exp(z_j)}}=\frac{e^{z_j}}{\sum\Omega}$$ with the sum is for all the different neurons in the softmax group  
With such outputs, when adding all possibilities of $y_i$, we get $1$ that is satisfied with the sum of all $y_i$ must be $1$, moreover, the outputs $y_i$ have to lie between $0$ and $1$ $\rightarrow$ use that softmax equation, we force the $y_i$ to represent a probability distribution over mutually exclusive alternatives.
- The softmax equation has a nice simple derivative: $$\frac{\partial y_i}{\partial z_i}=y_i(1-y_i)$$

Question for softmax equation:  
![question softmax](images/question_7.png?raw=True)
![question softmax 1](images/question_8.png?raw=True)
![question softmax 2](images/question_9.png?raw=True)
![question softmax 3](images/question_10.png?raw=True)

## 3.3. Cross-entropy: the right cost function to use with softmax
<a id="cross-enp"></a>
Now, the question is, if use the softmax group for the outputs, what's the right cost function?  
- The answer of the most appropriate cost function is the negative log probability of the correct answer 
$$C=-\sum_{j}{t_j}\ {log{y_j}}$$ is called cross-entropy cost function
![cross-entropy](images/cross-entropy.png?raw=True)  
- Aim: want to maximize the log probability of getting the right answer
- The derivative of cross-entropy w.r.t. the logit going into $i-th$ input
$$\frac{\partial C}{\partial z_i}=-\sum_{j=1}^C\frac{\partial t_jlog(y_j)}{\partial z_i}=\sum_{j}{\frac{\partial C}{\partial y_j}}\ \frac{\partial y_j}{\partial z_i}=\sum_{j=1}^C-t_j\frac{1}{y_j}\frac{\partial y_j}{\partial z_i}$$
$$=-\frac{t_i}{y_i}\frac{\partial y_i}{\partial z_i}-\sum_{j\neq i}^C{\frac{t_j}{y_j}\frac{\partial y_j}{\partial z_i}}=-\frac{t_i}{y_i}y_i(1-y_i)-\sum_{j\neq i}^C\frac{t_j}{y_j}(-y_jy_i)$$
$$=-t_i+t_iy_i+\sum_{j\neq i}^C{t_jy_i}=-t_i+\sum_{j=1}^C{t_jy_i}=-t_i+y_i\sum_{j=1}^Ct_j$$
$$=y_i-t_i$$
**Note** that we want to derived $\frac{\partial y_j}{\partial z_i}$ for $i=j$ and $i\neq j$, as the following:  
$$if\ i = j: \frac{\partial y_i}{\partial z_i}=\frac{\partial \frac{e^{z_i}}{\sum_{j\in group}{e^{z_j}}}}{\partial z_i}=\frac{e^{z_i}\sum\Omega-e^{z_i}e^{z_i}}{\sum^2\Omega}=\frac{e^{z_i}}{\sum\Omega}\ \frac{\sum\Omega-e^{z_i}}{\sum\Omega}=\frac{e^{z_i}}{\sum\Omega}(1-\frac{e^{z_i}}{\sum\Omega})=y_i(1-y_i)$$
  
$$if\ i\neq j: \frac{\partial y_i}{\partial z_j}=\frac{\partial \frac{e^{z_i}}{\sum\Omega}}{\partial z_j}=\frac{0-e^{z_i}e^{z_j}}{\sum^2\Omega}=-\frac{e^z_i}{\sum\Omega}\frac{e^{z_j}}{\sum\Omega}=-y_iy_j$$